In [1]:
import math

l09 = - math.log10(0.9)
l05 = - math.log10(0.5)
l03 = - math.log10(0.3)

def calc_rel_prob_divine(x):
    """update rel_prob given current divine information
    core for prediction
    """
    rel_lpdvn = 0.0
    # possessed might not divine werewolf as werewolf
    rel_lpdvn += x.dvn_cnt[2][1][1] * l09
    # werewolf might not divine possessed as werewolf
    rel_lpdvn += x.dvn_cnt[1][2][1] * l05
    # werewolf would not divine werewolf as werewolf
    rel_lpdvn += x.dvn_cnt[1][1][1] * 1
    # village Seer should not tell a lie
    rel_lpdvn += (x.dvn_cnt[0][0][1] + x.dvn_cnt[0][1][0] + x.dvn_cnt[0][2][1]) * 2
    return math.pow(10, -rel_lpdvn)

def calc_rel_prob_inquest(x):
    """update rel_prob given current inquest information
    core for prediction
    """
    # village Medium should not tell a lie
    return math.pow(0.01, x.inq_cnt[0][0][1] + x.inq_cnt[0][1][0] + x.inq_cnt[0][2][1])

def calc_rel_prob_vote(x):
    """update part of rel_prob given current vote information
    core for prediction
    """
    rel_lpvt = 0.0
    # werewolf might not vote possessed
    rel_lpvt += x.vote_cnt[1][2] * l09
    # possessed might not vote werewolf
    rel_lpvt += x.vote_cnt[2][1] * l09
    # werewolf would not vote werewolf
    rel_lpvt += x.vote_cnt[1][1] * l05
    return math.pow(10, -rel_lpvt)

def calc_rel_prob_attack(x):
    """update part of rel_prob given current attack information
    core for prediction
    """
    # werewolves are never to be attacked
    if x.atc_cnt[1] > 0:
        return 0.0
    else:
        return 1.0

def calc_rel_prob_co(x):
    """update part of rel_prob given current CO information
    core for prediction
    """
    rel_lpco = 0.0
    # village Seer would comingout
    if x.co_cnt[2][0] + x.co_cnt[1][0] > 0 and x.co_cnt[0][0] == 0:
        rel_lpco += 2.0
    # village Medium would comingout
    if x.co_cnt[2][1] + x.co_cnt[1][1] > 0 and x.co_cnt[0][1] == 0:
        rel_lpco += 2.0
    # TODO:bodyguard
    # possessed would comingout
    if sum(x.co_cnt[2]) == 0:
        rel_lpco += 1.0
    # werewolves might not comingout
    rel_lpco += (x.co_cnt[1][0] + x.co_cnt[1][1] + x.co_cnt[1][2]) * l05
    # villagers must not comingout
    if x.co_cnt[0][0] > 1:
        rel_lpco += 3.0
    if x.co_cnt[0][1] > 1:
        rel_lpco += 3.0
    if x.co_cnt[0][2] > 1:
        rel_lpco += 3.0
    return math.pow(10, -rel_lpco)

class EnemyCase(object):
    """object for each case who are werewolves and who is possessed
    get information and update fitness
    basic concept is Bayes' theorem : P(B|A) = P(B) * (P(A|B) / P(A))
    B: this case is true, A: Information
    rel_prob stands for P(B|A)
    """
    def __init__(self, ww, ps, vilsize):
        # P(B|A)
        self.rel_prob = 1.0
        self.rel_prob_co = 1.0
        self.rel_prob_vote = 1.0
        self.rel_prob_divine = 1.0
        self.rel_prob_inquest = 1.0
        self.rel_prob_attack = 1.0
        
        # member
        self.hm_idx = [idx for idx in range(1, vilsize+1) if idx != ps and idx not in ww]
        self.ww_idx = ww # werewolves
        self.ps_idx = [ps] # possessed
        self.dict_idx = dict()
        
        for idx in range(1, vilsize+1):
            if idx in self.hm_idx:
                self.dict_idx[idx] = 0
            elif idx in self.ww_idx:
                self.dict_idx[idx] = 1
            else:
                self.dict_idx[idx] = 2
        
        # vote
        # count, 0:villager, 1:werewolf, 2:possessed
        self.vote_cnt = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        # likelyhood
        # TODO 1
        
        # co 0:seer, 1:medium, 2:bodyguard
        self.co_cnt = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        
        # divine
        # count
        self.dvn_cnt = [[[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]]]
        # entropy
        # TODO 2
        
        # inquest
        # count
        self.inq_cnt = [[[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]]]
        # entropy
        # TODO?
        
        # attacked
        self.atc_cnt = [0, 0, 0]
    
    
    def initialize(self):
        # P(B|A)
        self.rel_prob = 1.0
        self.rel_prob_co = 1.0
        self.rel_prob_vote = 1.0
        self.rel_prob_divine = 1.0
        self.rel_prob_inquest = 1.0
        self.rel_prob_attack = 1.0
        
        # vote
        # count, 0:villager, 1:werewolf, 2:possessed
        self.vote_cnt = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        # likelyhood
        # TODO 1
        # co 0:seer, 1:medium, 2:bodyguard
        self.co_cnt = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        
        # divine
        # count
        self.dvn_cnt = [[[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]]]
        # entropy
        # TODO 2
        
        # inquest
        # count
        self.inq_cnt = [[[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]], [[0, 0], [0, 0], [0, 0]]]
        # entropy
        # TODO?
        
        # attacked
        self.atc_cnt = [0, 0, 0]
    
    def return_rel_prob(self):
        """return rel_prob
        """
        return self.rel_prob
    
    def update_rel_prob(self):
        """update rel_prob given current information
        """
        self.rel_prob = self.rel_prob_co * self.rel_prob_vote * self.rel_prob_divine * self.rel_prob_inquest * self.rel_prob_attack
    
    def show_status(self):
        """for debug
        returns currrent status
        """
        # todo
        pass
    
    def get_votelist(self, votelist):
        """get votelist as list of list [[x, y],...]
        x:agent, y:target
        """
        for vote in votelist:
            self.vote_cnt[self.dict_idx[vote[0]]][self.dict_idx[vote[1]]] += 1
        # update rel_prob_vote
        self.rel_prob_vote = calc_rel_prob_vote(self)
        rel_lpvt = 0.0
    
    def get_colist(self, colist):
        """get list of new co made from talklist in form [[x, role]]
        x:agent, role:role, 0:seer, 1:medium, 2:bodyguard
        """
        for co in colist:
            self.co_cnt[self.dict_idx[co[0]]][co[1]] += 1
        # update rel_prob_co
        self.rel_prob_co = calc_rel_prob_co(self)
    
    def get_divine_list(self, divinelist):
        """get new divine result list made from talklist in form [[x, y, race]]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        for divine in divinelist:
            self.dvn_cnt[self.dict_idx[divine[0]]][self.dict_idx[divine[1]]][divine[2]] += 1
        # update rel_prob_divine
        self.rel_prob_divine = calc_rel_prob_divine(self)
    
    def get_inquest_list(self, inquestlist):
        """get new inquest result list made from talklist in form [[x, y, race]]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        for inquest in inquestlist:
            self.inq_cnt[self.dict_idx[inquest[0]]][self.dict_idx[inquest[1]]][inquest[2]] += 1
        # update rel_prob_inquest
        self.rel_prob_inquest = calc_rel_prob_inquest(self)

    def get_attacked(self, x):
        """get agentid newly attacked by werewolves
        x:agent
        """
        self.atc_cnt[self.dict_idx[x]] += 1
        # update rel_prob_attack
        self.rel_prob_attack = calc_rel_prob_attack(self)

In [2]:
# pre_calculate
glocbal_import_case_list_13 = []
vilsize = 13
glocbal_import_case_list_13 = []
for ps in range(1, vilsize+1):
    for w1 in range(1,vilsize-1):
        for w2 in range(w1+1, vilsize):
            for w3 in range(w2+1, vilsize+1):
                if ps not in [w1, w2, w3]:
                    glocbal_import_case_list_13.append(EnemyCase([w1, w2, w3], ps, vilsize))

glocbal_import_case_list = []
vilsize = 15
glocbal_import_case_list = []
for ps in range(1, vilsize+1):
    for w1 in range(1,vilsize-1):
        for w2 in range(w1+1, vilsize):
            for w3 in range(w2+1, vilsize+1):
                if ps not in [w1, w2, w3]:
                    glocbal_import_case_list.append(EnemyCase([w1, w2, w3], ps, vilsize))


class EnemyCaseList(object):
    """
    """
    def __init__(self, vilsize=15):
        # main object
        if vilsize == 13:
            self.case_list = glocbal_import_case_list_13
        else:
            self.case_list = glocbal_import_case_list
    
    def initialize(self):
        for ec in self.case_list:
            ec.initialize()

    def update_rel_prob(self):
        """update rel_prob given current information
        """
        for ec in self.case_list:
            ec.update_rel_prob()

    # to return 1
    # each agent
    def prob_now(self, agent_idx):
        """return [P(agent_idx = VILLAGER), P(agent_idx = POSSESSED), P(agent_idx = WEREWOLF)]
        """
        rel_p_hm = sum([x.rel_prob for x in self.case_list if agent_idx in x.hm_idx])
        rel_p_ps = sum([x.rel_prob for x in self.case_list if agent_idx in x.ps_idx])
        rel_p_ww = sum([x.rel_prob for x in self.case_list if agent_idx in x.ww_idx])
        rel_psum = sum([x.rel_prob for x in self.case_list])
        return [rel_p_hm / rel_psum, rel_p_ps / rel_psum, rel_p_ww / rel_psum]
    
    # to return 2
    # each agent | Watashi Ninngen!
    def prob_now_wn(self, agent_idx, agent_idy):
        """return [P(agent_idx = VILLAGER), P(agent_idx = POSSESSED), P(agent_idx = WEREWOLF)]
        in condition that agent_idy = VILLAGER
        """
        rel_p_hm = sum([x.rel_prob for x in self.case_list if agent_idx in x.hm_idx and agent_idy in x.hm_idx])
        rel_p_ps = sum([x.rel_prob for x in self.case_list if agent_idx in x.ps_idx and agent_idy in x.hm_idx])
        rel_p_ww = sum([x.rel_prob for x in self.case_list if agent_idx in x.ww_idx and agent_idy in x.hm_idx])
        rel_psum = max(0.0000000001, sum([x.rel_prob for x in self.case_list if agent_idy in x.hm_idx]))
        return [rel_p_hm / rel_psum, rel_p_ps / rel_psum, rel_p_ww / rel_psum]
    # to return d1 (for debug)
    # prob for special case
    def prob_special_case(self, ww, ps):
        pel_p_spc = sum([x.rel_prob for x in self.case_list if ps in x.ps_idx and sorted(x.ww_idx) == sorted(ww)])
        rel_psum = sum([x.rel_prob for x in self.case_list])
        return  pel_p_spc / rel_psum

    def get_votelist(self, votelist):
        """get votelist as list of list [[x, y],...]
        x:agent, y:target
        """
        for ec in self.case_list:
            ec.get_votelist(votelist)

    def get_colist(self, colist):
        """get list of new co made from talklist in form [[x, role]]
        x:agent, role:role
        """
        for ec in self.case_list:
            ec.get_colist(colist)
        
    def get_divine_list(self, divinelist):
        """get new divine result list made from talklist in form [[x, y, race, p]]
        x:agent, y:target, race:"WEREWOLF" or "HUMAN", p=P(y is werewolf|x is human)
        """
        for ec in self.case_list:
            ec.get_divine_list(divinelist)

    def get_inquest_list(self, inquestlist):
        """get new inquest result list made from talklist in form [[x, y, race]]
        x:agent, y:target, race:"WEREWOLF" or "HUMAN"
        """
        for ec in self.case_list:
            ec.get_inquest_list(inquestlist)
        
    def get_attacked(self, x):
        """get agentid newly attacked by werewolves
        x:agent
        """
        for ec in self.case_list:
            ec.get_attacked(x)

In [3]:
game2_1 = EnemyCaseList(vilsize=13)
game2_1.initialize()

In [4]:
# もろもろ継承略

In [5]:
# 1日目
# 大浜と村中が占いCO
game2_1.get_colist([[4, 0], [9, 0]])
# 占い結果:大浜->森本(白), 村中->安西(白)
game2_1.get_divine_list([[4, 12, 0], [9, 10, 0]])
# 大野が霊媒CO
game2_1.get_colist([[11, 1]])
# 投票
game2_1.get_votelist([[8, 6], [4, 9], [2, 6], [12, 13], [5, 8], [6, 11], [3, 5], [9, 8], [13, 11], [11, 13], [1, 5], [10, 8], [7, 11]])
# 再投票
game2_1.get_votelist([[1, 8], [2, 8], [3, 11], [4, 8], [5, 8], [6, 8], [7, 11], [9, 8], [10, 8], [12, 8], [13, 11]])


In [6]:
# 2日目
# 襲撃:なし
# デイジーと児玉が霊媒CO
game2_1.get_colist([[3, 1], [7, 1]])
# 霊媒結果:デイジー->中田(白), 児玉->中田(白), 大野->中田(白)
game2_1.get_inquest_list([[3, 8, 0], [7, 8, 0], [11, 8, 0]])
# 占い結果:大浜->デイジー(白), 村中->児玉(白)
game2_1.get_divine_list([[4, 3, 0], [9, 7, 0]])
# 投票
game2_1.get_votelist([[11, 7], [3, 7], [1, 11], [2, 11], [7, 11], [5, 7], [6, 11], [13, 11], [12, 7], [4, 7], [10, 7], [9, 11]])
# 再投票
game2_1.get_votelist([[1, 11], [2, 11], [3, 7], [4, 7], [5, 7], [6, 11], [9, 7], [10, 11], [12, 7], [13, 7]])


In [7]:
# 3日目
# 襲撃:メイソン
game2_1.get_attacked(1)
# 霊媒結果:デイジー->児玉(黒), 大野->児玉(黒)
game2_1.get_inquest_list([[3, 7, 1], [11, 7, 1]])
# 占い結果:大浜->メイソン(白), 村中->メイソン(白)
game2_1.get_divine_list([[4, 1, 0], [9, 1, 0]])
# 投票
game2_1.get_votelist([[4, 9], [3, 9], [2, 11], [9, 11], [12, 11], [5, 3], [11, 9], [13, 11], [10, 11], [6, 11]])


In [8]:
# 4日目
# 襲撃:イシイ
game2_1.get_attacked(13)
# 霊媒結果:デイジー->大野(黒)
game2_1.get_inquest_list([[3, 11, 1]])
# 占い結果:大浜->ダンカン(白), 村中->森本(白)
game2_1.get_divine_list([[4, 2, 0], [9, 12, 0]])
# 投票
game2_1.get_votelist([[6, 3], [2, 3], [5, 3], [4, 5], [3, 9], [10, 9], [9, 3], [12, 3]])


In [9]:
# 5日目
# 襲撃:森本
game2_1.get_attacked(12)
# 占い結果:大浜->武中(白), 村中->武中(黒)
game2_1.get_divine_list([[4, 5, 0], [9, 5, 1]])
# 投票
game2_1.get_votelist([[5, 9], [9, 5], [4, 9], [2, 5], [6, 5], [10, 5]])


In [10]:
# 6日目
# 襲撃:ダンカン
game2_1.get_attacked(2)
# 占い結果:大浜->香川(白), 村中->大浜(黒)
game2_1.get_divine_list([[4, 6, 0], [9, 4, 1]])


In [11]:
# 1:メイソン
# 2:ダンカン
# 3:デイジー
# 4:大浜
# 5:武中
# 6:香川
# 7:児玉
# 8:中田
# 9:村中
# 10:安西
# 11:大野
# 12:森本
# 13:イシイ

In [12]:
# 最終日の投票前の時点で判断
game2_1.update_rel_prob()

In [13]:
# 大浜目線

In [14]:
game2_1.prob_now_wn(6, 4)

[0.9699116078158678, 0.0014304554934660538, 0.02865793669066576]

In [15]:
game2_1.prob_now_wn(9, 4)

[6.0680676427002736e-09, 0.06682154136671418, 0.9331784525652184]

In [16]:
game2_1.prob_now_wn(10, 4)

[0.05355241159906665, 0.001427401881675844, 0.945020186519257]

In [17]:
# 香川目線

In [18]:
game2_1.prob_now_wn(4, 6)

[0.19300883307251154, 0.001257333115061165, 0.8057338338124264]

In [19]:
game2_1.prob_now_wn(9, 6)

[0.8047200343375566, 0.015376405637803599, 0.17990356002463934]

In [20]:
game2_1.prob_now_wn(10, 6)

[0.8110411020600857, 0.0007044690929703134, 0.18825442884694316]

In [21]:
# 村中目線

In [22]:
game2_1.prob_now_wn(4, 9)

[1.4865409434096901e-09, 0.001442441974793559, 0.9985575565386647]

In [23]:
game2_1.prob_now_wn(6, 9)

[0.9906636950089991, 0.00037763272603872006, 0.008958672264962683]

In [24]:
game2_1.prob_now_wn(10, 9)

[0.9992369929618349, 0.000517591217193497, 0.00024541582097133136]

In [25]:
# 安西目線

In [26]:
game2_1.prob_now_wn(4, 10)

[0.012923329173671767, 0.001528677119646118, 0.985547993706682]

In [27]:
game2_1.prob_now_wn(6, 10)

[0.9835426777052854, 0.0007168544781470394, 0.01574046781656796]

In [28]:
game2_1.prob_now_wn(9, 10)

[0.9843225063742381, 0.003810850463600946, 0.011866643162160692]